In [ ]:
import torch
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import integrate
from scipy import stats

home = Path.home()
results = home/'group'/'project'/'results'
snr_sweep = np.linspace(1, 10, 19)


# plot pdf vs snr for matched filter

In [ ]:
sns.set_theme()
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

x_pdf = np.linspace(0, 20, 1001)

for snr in [1.0, 2.0, 3.0, 5.0, 10.0]:
    
    mf_pdf = stats.rice.pdf(
        x_pdf,
        snr,
        loc=snr * (1 - 1 / math.sqrt(2)),
        scale=1/math.sqrt(2)
    )
    
    ax.plot(x_pdf, mf_pdf)
    
ax.plot(x_pdf, stats.rayleigh.pdf(x_pdf, loc=0, scale=1/math.sqrt(2)))

# Load dnn results

In [ ]:
dnn_results = results/'machine_learning'/'dnn'/'triggering'/'eval_outputs'/'toy_sinusoid'/'220223_snr_sweep'/'220224_cnn_pdf_and_cdf_vs_snr.npz'
dnn_results = np.load(dnn_results)

# compare efficiency vs snr for select tpr

In [ ]:
# matched filter
select_fpr = 1e-2
ind_fpr = np.argmin(abs(stats.rayleigh.sf(x_pdf, loc=0, scale=1/math.sqrt(2)) - select_fpr))

mf_efficiency = np.zeros(dnn_results['snr'].size)
dnn_efficiency = np.zeros(dnn_results['snr'].size)

for i, snr in enumerate(dnn_results['snr']):
    
    sf = stats.rice.sf(
        x_pdf,
        snr,
        loc=snr * (1 - 1 / math.sqrt(2)),
        scale=1/math.sqrt(2)
    )
    
    mf_efficiency[i] = sf[ind_fpr]
    
# dnn

for i, snr in enumerate(dnn_results['snr']):
    ind_fpr = np.argmin(abs(1 - dnn_results['cdf_noise'][i, :] - select_fpr))
    dnn_efficiency[i] = 1- dnn_results['cdf_signal'][i, ind_fpr]


In [ ]:
sns.set_theme()
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(dnn_results['snr'], mf_efficiency)
ax.plot(dnn_results['snr'], dnn_efficiency)

In [ ]:
sns.set_theme(context='talk')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

clist = sns.color_palette('deep', n_colors=4)
fpr_list = [1e-1, 1e-2, 1e-3, 1e-4]

for n, select_fpr in enumerate(fpr_list):
    
    ind_fpr = np.argmin(abs(stats.rayleigh.sf(x_pdf, loc=0, scale=1/math.sqrt(2)) - select_fpr))

    mf_efficiency = np.zeros(dnn_results['snr'].size)
    dnn_efficiency = np.zeros(dnn_results['snr'].size)

    for i, snr in enumerate(dnn_results['snr']):

        sf = stats.rice.sf(
            x_pdf,
            snr,
            loc=snr * (1 - 1 / math.sqrt(2)),
            scale=1/math.sqrt(2)
        )

        mf_efficiency[i] = sf[ind_fpr]

    for i, snr in enumerate(dnn_results['snr']):
        ind_fpr = np.argmin(abs(1 - dnn_results['cdf_noise'][i, :] - select_fpr))
        dnn_efficiency[i] = 1- dnn_results['cdf_signal'][i, ind_fpr]
        
        
    ax.plot(dnn_results['snr'], mf_efficiency, color=clist[n], linestyle='-', label=f'MF, FPR = {select_fpr}')
    ax.plot(dnn_results['snr'], dnn_efficiency, color=clist[n], linestyle='--', label=f'DNN, FPR = {select_fpr}')

line_mf, = ax.plot([], [], linestyle='-', color='k', label='MF')
line_dnn, = ax.plot([], [], linestyle='--', color='k', label='DNN')
line_color_list = []
for i, color in enumerate(clist):
    line, = ax.plot([], [], linestyle='-', color=color, label=f'FPR={fpr_list[i]}')
    line_color_list.append(line)
    
ax.legend(handles=[line_mf, line_dnn, *line_color_list], loc=4)
ax.set_xlabel('Signal SNR')
ax.set_ylabel('Efficiency')
ax.set_title('Efficiency vs Signal SNR')

plt.tight_layout()
name = '220224_dnn_vs_mf_efficiency_test_signal_snr.png'
save_path = Path.home()/'group'/'project'/'plots'/'machine_learning'/'dnn'/'triggering'/'toy_sinusoid'/name
plt.savefig(save_path)

    